In [3]:
# setting the environment variables
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

from config import set_environment
set_environment()

# LLMs

In [4]:
from langchain_openai import OpenAI

llm = OpenAI()

In [6]:
from langchain_anthropic import AnthropicLLM

llm = AnthropicLLM(model='claude-2.1')

In [7]:
summary = llm.invoke("Tell me a joke about light bulbs!")

In [10]:
print(summary)



Why did the light bulb go to therapy?

Because it was feeling a little dim!


In [11]:
from langchain.llms import HuggingFaceHub
llm = HuggingFaceHub(
    model_kwargs={"temperature": 0.5, "max_length": 64},
    repo_id="google/flan-t5-xxl"
)
prompt = "In which country is Tokyo?"
completion = llm(prompt)
print(completion)

/Users/ben/anaconda3/envs/newgen/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/Users/ben/anaconda3/envs/newgen/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


japan


In [12]:
from langchain.llms.fake import FakeListLLM
fake_llm = FakeListLLM(responses=["Hello"])
fake_llm.invoke("Hi and goodbye, FakeListLLM!")

'Hello'

# Chat models

In [13]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(model_name='gpt-4-0613')
response = llm.invoke([HumanMessage(content='Say "Hello world" in Python.')])
print(response)


content='print("Hello world")' response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 15, 'total_tokens': 20}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}


In [17]:
from langchain.schema import SystemMessage

chat_output = llm.invoke([ 
    SystemMessage(content="You're a helpful assistant"), 
    HumanMessage(content="What is the purpose of model regularization?") 
])
print(chat_output)

content='Model regularization is a technique used to prevent overfitting in machine learning models. Overfitting happens when a model learns too much from the training data, including noise and outliers, and performs poorly on unseen data.\n\nRegularization works by adding a penalty term to the loss function, which discourages the learning algorithm from assigning too much importance to any individual predictor (feature). This effectively reduces the complexity of the model, making it more generalizable to unseen data.\n\nThere are different types of regularization techniques, including L1 regularization (Lasso regression), L2 regularization (Ridge regression), and a combination of both (Elastic net). \n\nIn summary, the purpose of model regularization is to improve the generalization capability of a model, reduce model complexity, and prevent overfitting.' response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 24, 'total_tokens': 181}, 'model_name': 'gpt-4-0613'

# Prompts

In [20]:
prompt = """
Summarize this text in one sentence:

{text}
"""

llm = OpenAI()
summary = llm(prompt.format(text="Some long story"))


In [21]:
print(summary)


The text is a long story.


In [22]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.") 


In [23]:
formatted_prompt = prompt_template.format(adjective="funny", content="chickens") 

In [24]:
print(formatted_prompt)

Tell me a funny joke about chickens.


In [25]:
prompt_val = prompt_template.invoke({"adjective": "funny", "content": "chickens"}) 

In [26]:
prompt_val

StringPromptValue(text='Tell me a funny joke about chickens.')

In [27]:
from langchain_openai.chat_models import ChatOpenAI 
from langchain.prompts import ChatPromptTemplate
# Define a ChatPromptTemplate to translate text
template = ChatPromptTemplate.from_messages([
    ('system', 'You are an English to French translator.'),
    ('user', 'Translate this to French: {text}')
])
llm = ChatOpenAI()
# Translate a joke about light bulbs
response = llm.invoke(template.format_messages(text='How many programmers does it take to change a light bulb?'))


In [28]:
print(response)

content='Combien de programmeurs faut-il pour changer une ampoule?' response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 36, 'total_tokens': 50}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}


In [33]:
from langchain_google_vertexai import ChatVertexAI
#from langchain_google_genai import GoogleGenerativeAI

from langchain import PromptTemplate, LLMChain

llm = ChatVertexAI(model_name="gemini-pro")
#llm = GoogleGenerativeAI(model="gemini-pro")

template = """Question: {question}
Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"
llm_chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:
Question: What NFL team won the Super Bowl in the year Justin Beiber was born?
Answer: Let's think step by step.

> Finished chain.


'**Step 1: Find the year Justin Bieber was born.**\n\nJustin Bieber was born on March 1, 1994.\n\n**Step 2: Determine the NFL season that began in 1993 and ended in 1994.**\n\nThis would be the 1993 NFL season.\n\n**Step 3: Find out which NFL team won the Super Bowl at the end of the 1993 NFL season.**\n\nThe Dallas Cowboys won Super Bowl XXVIII at the end of the 1993 NFL season.\n\n**Answer: Dallas Cowboys**'

# LangChain Expression Language

In [85]:
from langchain_community.llms import HuggingFaceEndpoint 
from langchain.prompts import PromptTemplate 
from langchain.schema import StrOutputParser
repo_id = "mistralai/Mistral-7B-Instruct-v0.2" 
llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=128, temperature=0.5,
)
template = """Question: {question} Answer: Let's think step by step.""" 
prompt = PromptTemplate.from_template(template) 
runnable = prompt | llm | StrOutputParser()


                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/ben/.cache/huggingface/token
Login successful


In [40]:
question = "Who won the FIFA World Cup in the year 1994? " 
summary = runnable.invoke({"question": question})

In [42]:
print(summary)

 The FIFA World Cup is an international football competition. It is held every four years. So, to find out which team won the FIFA World Cup in 1994, we need to identify which World Cup tournament that was.

First, we know that the first FIFA World Cup was held in 1930. Since then, the tournament has been held every four years, except for 1942 and 1946 due to World War II.

To calculate which World Cup tournament was held in 1994, we can subtract the year of the first World Cup (1930) from 1994 and divide by 4. We'll round down to the nearest whole number because the tournament is held in the year indicated, not the year before.

1994 - 1930 = 64
64 / 4 = 16.025

Since we can't have a fraction of a tournament, we round down to 16. That means there have been 16 World Cup tournaments as of 1994. The most recent one before 1994 was in 1990. So, the FIFA World Cup in 1994 would be the one following that, which was won by Brazil.

Therefore, the answer is: Brazil won the FIFA World Cup in 1

# Decorator syntax

In [53]:
from langchain_decorators import llm_prompt

@llm_prompt
def summarize(topic: str, manner: str = "fun") -> str:
    """
    Write a short paragraph in {manner} fashion about {topic}
    """
    return

summary = summarize(topic="The history of Python is both interesting and entertaining.")


In [54]:
print(summary)

The history of Python is like a wild and wacky adventure through the coding jungle. It all started back in the late 1980s when a brilliant programmer named Guido van Rossum decided to create a language that was easy to read and fun to use. As Python evolved, it gained a cult following of developers who were drawn to its simplicity and versatility. With its quirky name and playful spirit, Python has become a beloved language in the world of programming. So, grab your snake charmer and get ready to slither through the fascinating history of Python!


# Text-To-Image

In [65]:
from langchain.chains import LLMChain 
from langchain.prompts import PromptTemplate 
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper 
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9) 
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template=(
        "Generate a concise prompt to generate an image based on the following description: "
        "{image_desc}"
))
chain = LLMChain(llm=llm, prompt=prompt)

In [70]:
# prompt = chain.run("halloween night at a haunted museum")
# print(prompt)

In [66]:
image_url = DallEAPIWrapper().run(chain.run("halloween night at a haunted museum"))

In [67]:
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-v2zpDq7lmzeE7QnOAKN5KtVD/user-1kc8PP1odvcm9qydlgvraAFz/img-Pn9wx3ImDdNoHSJaKR9gYmHu.png?st=2024-04-04T08%3A15%3A26Z&se=2024-04-04T10%3A15%3A26Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-04-03T22%3A36%3A11Z&ske=2024-04-04T22%3A36%3A11Z&sks=b&skv=2021-08-06&sig=tHlsFm990prI/PuqtgmjgIsr8UYkYGoEFw08sHrT7/Q%3D'

In [68]:
from IPython.display import Image, display
display(Image(url=image_url))


In [69]:
import requests
response = requests.get(image_url)
image_path = "haunted_house.png"
with open(image_path, "wb") as f:
    f.write(response.content)


In [74]:
from langchain_community.llms import Replicate
text2image = Replicate(
    model=(
        "stability-ai/stable-diffusion:"
        "d70beb400d223e6432425a5299910329c6050c6abcf97b8c70537d6a1fcb269a"
    ),
    model_kwargs={"image_dimensions": "512x512"}
)
image_url = text2image("a book cover for a book about creating generative ai applications in Python")


In [76]:
from IPython.display import Image, display
display(Image(url=image_url))

In [79]:
from langchain_core.messages import HumanMessage, SystemMessage 
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=256)
chat.invoke([
    HumanMessage(
        content=[
            {"type": "text", "text": "What is this image showing"}, 
            {
                "type": "image_url", 
                "image_url": { "url": image_url, "detail": "auto", },
            }, 
        ]
    )
])


AIMessage(content="This image appears to be a digital or stylized representation of an object or pattern, possibly intended to be abstract or artistic in nature. The central figure is a hexagon-like shape made up of smaller hexagons in a blue color, with yellow dots placed at the vertices of the smaller hexagons. The background is a vibrant orange with a green circular border, and the whole image is overlaid with a texture that gives it a somewhat crackled or mosaic appearance.\n\nAdditionally, there is text overlaying the image in a script that is difficult to read due to the decorative font and the complex background. The text doesn't seem to correspond to any recognizable language and may be purely aesthetic or fictional. The image might be a cover for a music album, a piece of artwork, or a representation of something from a specific subculture or fandom, but without more context, it's hard to determine its exact purpose or origin.", response_metadata={'token_usage': {'completion_t

In [81]:
langchain_image = """
The image appears to be a diagram representing the architecture or components of a software system or framework related to language processing, possibly named LangChain or associated with a project or product called LangChain, based on the prominent appearance of that term. The diagram is organized into several layers or aspects, each containing various elements or modules:\n\n1. **Protocol**: This may be the foundational layer, which includes "LCEL" and terms like parallelization, fallbacks, tracing, batching, streaming, async, and composition. These seem related to communication and execution protocols for the system.\n\n2. **Integrations Components**: This layer includes "Model I/O" with elements such as the model, output parser, prompt, and example selector. It also has a "Retrieval" section with a document loader, retriever, embedding model, vector store, and text splitter. Lastly, there\'s an "Agent Tooling" section. These components likely deal with the interaction with external data, models, and tools.\n\n3. **Application**: The application layer features "LangChain" with chains, agents, agent executors, and common application logic. This suggests that the system uses a modular approach with chains and agents to process language tasks.\n\n4. **Deployment**: This contains "Lang'"
"""

In [82]:
# prompt = chain.run("halloween night at a haunted museum")
# print(prompt)
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template=(
        "Simplify this image description into a concise prompt to generate an image: "
        "{image_desc}"
))
chain = LLMChain(llm=llm, prompt=prompt)
prompt = chain.run(langchain_image)
print(prompt)


"Design a software system or framework for language processing called LangChain, with layers for protocol, integration components, application, and deployment. The system uses chains and agents for processing and includes components for communication protocols, external data interaction, and common application logic."


In [83]:
image_url = DallEAPIWrapper().run(prompt)

In [84]:
from IPython.display import Image, display
display(Image(url=image_url))